# Processamento de Dados de Emissões de GEE para a Serra do Penitente
Este notebook processa os dados de emissões de GEE (Gases de Efeito Estufa) para os municípios da Serra do Penitente com base nos dados do SEEG.

In [1]:
# Importar bibliotecas necessárias
import pandas as pd
import os

In [2]:
# Criar diretório de saída se não existir
os.makedirs("data/generated", exist_ok=True)

In [3]:
# Caminho local do arquivo SEEG (formato .xlsx)
seeg_path = "data/SEEG-12-dados-municipais.xlsx"

In [4]:
# Municípios da Serra do Penitente
municipios_alvo = ["Balsas (MA)", "Tasso Fragoso (MA)", "Alto Parnaíba (MA)"]

In [5]:
# Carregar planilha correta (ajustar nome se necessário)
df = pd.read_excel(seeg_path, sheet_name="Dados")
print(df.head())

  Emissão/Remoção/Bunker Setor de emissão Categoria emissora  \
0                Emissão     Agropecuária   Cultivo de arroz   
1                Emissão     Agropecuária   Cultivo de arroz   
2                Emissão     Agropecuária   Cultivo de arroz   
3                Emissão     Agropecuária   Cultivo de arroz   
4                Emissão     Agropecuária   Cultivo de arroz   

                 Sub-categoria emissora  Recorte Atividade geral Estado  \
0  Cultivo em sistema irrigado inundado  Diretas     Agricultura   Acre   
1  Cultivo em sistema irrigado inundado  Diretas     Agricultura   Acre   
2  Cultivo em sistema irrigado inundado  Diretas     Agricultura   Acre   
3  Cultivo em sistema irrigado inundado  Diretas     Agricultura   Acre   
4  Cultivo em sistema irrigado inundado  Diretas     Agricultura   Acre   

           Município  ID Território  1970  ...  2014  2015  2016  2017  2018  \
0    Acrelândia (AC)       11200013   0.0  ...   0.0   0.0   0.0   0.0   0.0   
1  A

In [6]:
# Filtrar apenas os municípios desejados
df_municipios = df[df["Município"].isin(municipios_alvo)]
print(df_municipios.head())

     Emissão/Remoção/Bunker Setor de emissão    Categoria emissora  \
1136                Emissão     Agropecuária      Cultivo de arroz   
1151                Emissão     Agropecuária      Cultivo de arroz   
1329                Emissão     Agropecuária      Cultivo de arroz   
6706                Emissão     Agropecuária  Fermentação entérica   
6721                Emissão     Agropecuária  Fermentação entérica   

                          Sub-categoria emissora  Recorte Atividade geral  \
1136        Cultivo em sistema irrigado inundado  Diretas     Agricultura   
1151        Cultivo em sistema irrigado inundado  Diretas     Agricultura   
1329        Cultivo em sistema irrigado inundado  Diretas     Agricultura   
6706  Processo de digestão de animais ruminantes  Diretas        Pecuária   
6721  Processo de digestão de animais ruminantes  Diretas        Pecuária   

        Estado           Município  ID Território          1970  ...  \
1136  Maranhão  Alto Parnaíba (MA)       121

In [7]:
# Manter apenas anos válidos
df_long = df_municipios.melt(
    id_vars=["Município", "Estado"],
    var_name="Ano",
    value_name="GEE_tCO2e"
)
print(df_long.head())

            Município    Estado                     Ano GEE_tCO2e
0  Alto Parnaíba (MA)  Maranhão  Emissão/Remoção/Bunker   Emissão
1         Balsas (MA)  Maranhão  Emissão/Remoção/Bunker   Emissão
2  Tasso Fragoso (MA)  Maranhão  Emissão/Remoção/Bunker   Emissão
3  Alto Parnaíba (MA)  Maranhão  Emissão/Remoção/Bunker   Emissão
4         Balsas (MA)  Maranhão  Emissão/Remoção/Bunker   Emissão


In [8]:
# Transformar colunas de ano em formato long
df_long = df_long[df_long["Ano"].astype(str).str.isnumeric()]
df_long["Ano"] = df_long["Ano"].astype(int)
df_long["GEE_tCO2e"] = pd.to_numeric(df_long["GEE_tCO2e"], errors="coerce")
print(df_long.head())

               Município    Estado   Ano     GEE_tCO2e
1400  Alto Parnaíba (MA)  Maranhão  1970     40.694441
1401         Balsas (MA)  Maranhão  1970     82.436357
1402  Tasso Fragoso (MA)  Maranhão  1970     29.329327
1403  Alto Parnaíba (MA)  Maranhão  1970  39123.239960
1404         Balsas (MA)  Maranhão  1970  39654.606320



In [9]:
# Agrupar total por município-ano
df_grouped = df_long.groupby(["Município", "Ano"], as_index=False)["GEE_tCO2e"].sum()
print(df_grouped.head())

            Município   Ano     GEE_tCO2e
0  Alto Parnaíba (MA)  1970  52853.898162
1  Alto Parnaíba (MA)  1971  54970.857021
2  Alto Parnaíba (MA)  1972  57236.913124
3  Alto Parnaíba (MA)  1973  59529.418663
4  Alto Parnaíba (MA)  1974  61792.350424


In [10]:
# Salvar o dataset final
df_grouped.to_csv("data/generated/gee_serra_penitente.csv", index=False)
print("✅ Dataset gerado: data/generated/gee_serra_penitente.csv")

✅ Dataset gerado: data/generated/gee_serra_penitente.csv


In [12]:
# print(df_grouped.head(10))
display(df_grouped.head(10))

,Município,Ano,GEE_tCO2e
0,Alto Parnaíba (MA),1970,52853.898162
1,Alto Parnaíba (MA),1971,54970.857021
2,Alto Parnaíba (MA),1972,57236.913124
3,Alto Parnaíba (MA),1973,59529.418663
4,Alto Parnaíba (MA),1974,61792.350424
5,Alto Parnaíba (MA),1975,72835.033601
6,Alto Parnaíba (MA),1976,77885.193454
7,Alto Parnaíba (MA),1977,80968.095115
8,Alto Parnaíba (MA),1978,85308.073464
9,Alto Parnaíba (MA),1979,80624.422426
